In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [2]:
# read csv data
df_toronto_review = pd.read_csv('../data/toronto_reviews.csv')
df_business = pd.read_csv('../data/businesses_final_toronto.csv')

In [3]:
# rename business id
df_business.rename(columns={'id': 'business_id'}, inplace=True)

In [4]:
df_business.head(2)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAb...,False,"{'address1': '61 Ossington Avenue', 'address2'...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto...
1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 43.67212500034, 'longitude': -79....",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZ...,False,"{'address1': '2345 Queen Street E', 'address2'...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto...


In [5]:
print("total business numbers:", len(df_business))

total business numbers: 32123


In [6]:
restaurantString = "restaurant|Bar|Bakeries|food|Coffee|Tea|Waffles|Desserts\
|sandwiches|Cupcakes|cafes|Breakfast|Brunch|Lunch|Dinner|chicken|wings\
|singaporean|malaysian|indonesian|russian|taiwanese|american|chinese|vietnamese|moroccan\
|cantonese|caribbean|mideastern|asian|italian|korean|himalayan|african|nepalese|laotian|sushi|hawaiian\
|mediterranean|caterers|fish|chips|fishnchips|thai|salad|seafood|irish|portuguese|french|vegan|raw_food\
|pizza|latin|lebanese|burgers|gluten-free|japanese|pakistani|mexican|beer|sushi|venues|argentine|lounges|barbeque\
|canadian|dimsum|halal|afghan|donuts|juice|smoothies|bagels|ramen|icecream|pubs|falafel\
|noodles|soup|cheese|colombian|turkish|ethiopian|peruvian|hungarian|ukrainian|indian\
|persian|brazilian|vegetarian|austrian|arabian|haitian|cambodian|german|greek"

# We only take restaurant business
cond_category_restaurant = df_business["categories"].str.contains(restaurantString,case=False)

df_business_Restaurant = df_business[cond_category_restaurant]
print('size: ', len(df_business_Restaurant))

size:  10301


In [7]:
cond_category_Not_restaurant = ~df_business["categories"].str.contains(restaurantString,case=False)
business_not_res = df_business[cond_category_Not_restaurant]

In [8]:
# for index in range(0,100):
#     print(business_not_res['categories'][index])
#business_not_res['categories']

for index, value in business_not_res['categories'].items():
    print("Index : {}, Value : {}".format(index, value))

Index : 3, Value : [{'alias': 'healthtrainers', 'title': 'Trainers'}, {'alias': 'gyms', 'title': 'Gyms'}]
Index : 4, Value : [{'alias': 'locksmiths', 'title': 'Keys & Locksmiths'}]
Index : 6, Value : [{'alias': 'privatetutors', 'title': 'Private Tutors'}]
Index : 7, Value : [{'alias': 'medcenters', 'title': 'Medical Centers'}, {'alias': 'hospitals', 'title': 'Hospitals'}]
Index : 8, Value : [{'alias': 'chiropractors', 'title': 'Chiropractors'}, {'alias': 'massage_therapy', 'title': 'Massage Therapy'}, {'alias': 'naturopathic', 'title': 'Naturopathic/Holistic'}]
Index : 9, Value : [{'alias': 'movers', 'title': 'Movers'}, {'alias': 'selfstorage', 'title': 'Self Storage'}]
Index : 10, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}, {'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 12, Value : [{'alias': 'shopping', 'title': 'Shopping'}, {'alias': 'eventplanning', 'title': 'Party & Event Planning'}]
Index : 13, Value : [{'alias': 'acupuncture', 'title': 'Acupuncture'}, {'

Index : 83, Value : [{'alias': 'hair', 'title': 'Hair Salons'}]
Index : 86, Value : [{'alias': 'optometrists', 'title': 'Optometrists'}, {'alias': 'laserlasikeyes', 'title': 'Laser Eye Surgery/Lasik'}, {'alias': 'opticians', 'title': 'Eyewear & Opticians'}]
Index : 87, Value : [{'alias': 'copyshops', 'title': 'Printing Services'}, {'alias': 'couriers', 'title': 'Couriers & Delivery Services'}, {'alias': 'shipping_centers', 'title': 'Shipping Centers'}]
Index : 89, Value : [{'alias': 'contractors', 'title': 'Contractors'}]
Index : 90, Value : [{'alias': 'hobbyshops', 'title': 'Hobby Shops'}, {'alias': 'comicbooks', 'title': 'Comic Books'}, {'alias': 'bookstores', 'title': 'Bookstores'}]
Index : 91, Value : [{'alias': 'skatingrinks', 'title': 'Skating Rinks'}]
Index : 92, Value : [{'alias': 'limos', 'title': 'Limos'}, {'alias': 'airport_shuttles', 'title': 'Airport Shuttles'}]
Index : 94, Value : [{'alias': 'condominiums', 'title': 'Condominiums'}]
Index : 95, Value : [{'alias': 'copysho

Index : 188, Value : [{'alias': 'dentalhygienists', 'title': 'Dental Hygienists'}, {'alias': 'generaldentistry', 'title': 'General Dentistry'}, {'alias': 'cosmeticdentists', 'title': 'Cosmetic Dentists'}]
Index : 189, Value : [{'alias': 'homecleaning', 'title': 'Home Cleaning'}, {'alias': 'officecleaning', 'title': 'Office Cleaning'}]
Index : 190, Value : [{'alias': 'chiropractors', 'title': 'Chiropractors'}, {'alias': 'acupuncture', 'title': 'Acupuncture'}, {'alias': 'massage', 'title': 'Massage'}]
Index : 192, Value : [{'alias': 'electronics', 'title': 'Electronics'}, {'alias': 'securitysystems', 'title': 'Security Systems'}]
Index : 193, Value : [{'alias': 'securitysystems', 'title': 'Security Systems'}]
Index : 194, Value : [{'alias': 'accountants', 'title': 'Accountants'}]
Index : 195, Value : [{'alias': 'medicalspa', 'title': 'Medical Spas'}, {'alias': 'laser_hair_removal', 'title': 'Laser Hair Removal'}]
Index : 197, Value : [{'alias': 'libraries', 'title': 'Libraries'}]
Index :

Index : 289, Value : [{'alias': 'generaldentistry', 'title': 'General Dentistry'}, {'alias': 'oralsurgeons', 'title': 'Oral Surgeons'}, {'alias': 'cosmeticdentists', 'title': 'Cosmetic Dentists'}]
Index : 290, Value : [{'alias': 'furniture', 'title': 'Furniture Stores'}, {'alias': 'officeequipment', 'title': 'Office Equipment'}]
Index : 291, Value : [{'alias': 'adultedu', 'title': 'Adult Education'}, {'alias': 'careercounseling', 'title': 'Career Counseling'}, {'alias': 'language_schools', 'title': 'Language Schools'}]
Index : 292, Value : [{'alias': 'healthtrainers', 'title': 'Trainers'}, {'alias': 'gyms', 'title': 'Gyms'}, {'alias': 'sports_clubs', 'title': 'Sports Clubs'}]
Index : 294, Value : [{'alias': 'generaldentistry', 'title': 'General Dentistry'}]
Index : 295, Value : [{'alias': 'wholesale_stores', 'title': 'Wholesale Stores'}]
Index : 297, Value : [{'alias': 'sewingalterations', 'title': 'Sewing & Alterations'}]
Index : 298, Value : [{'alias': 'roofing', 'title': 'Roofing'}]

Index : 380, Value : [{'alias': 'vintage', 'title': 'Used, Vintage & Consignment'}, {'alias': 'accessories', 'title': 'Accessories'}]
Index : 381, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}, {'alias': 'tires', 'title': 'Tires'}]
Index : 383, Value : [{'alias': 'othersalons', 'title': 'Nail Salons'}, {'alias': 'eyelashservice', 'title': 'Eyelash Service'}, {'alias': 'waxing', 'title': 'Waxing'}]
Index : 384, Value : [{'alias': 'flooring', 'title': 'Flooring'}, {'alias': 'carpeting', 'title': 'Carpeting'}]
Index : 385, Value : [{'alias': 'apartments', 'title': 'Apartments'}]
Index : 386, Value : [{'alias': 'limos', 'title': 'Limos'}]
Index : 389, Value : [{'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 391, Value : [{'alias': 'headshops', 'title': 'Head Shops'}, {'alias': 'accessories', 'title': 'Accessories'}]
Index : 392, Value : [{'alias': 'petstore', 'title': 'Pet Stores'}]
Index : 393, Value : [{'alias': 'waxing', 'title': 'Waxing'}, {'alias': 'threadingservi

...

Index : 29267, Value : [{'alias': 'jewelry', 'title': 'Jewelry'}]
Index : 29269, Value : [{'alias': 'gyms', 'title': 'Gyms'}, {'alias': 'healthtrainers', 'title': 'Trainers'}]
Index : 29270, Value : [{'alias': 'insurance', 'title': 'Insurance'}]
Index : 29271, Value : [{'alias': 'contractors', 'title': 'Contractors'}]
Index : 29275, Value : [{'alias': 'servicestations', 'title': 'Gas Stations'}]
Index : 29276, Value : [{'alias': 'mobilephones', 'title': 'Mobile Phones'}]
Index : 29277, Value : [{'alias': 'localservices', 'title': 'Local Services'}]
Index : 29278, Value : [{'alias': 'hair', 'title': 'Hair Salons'}]
Index : 29281, Value : [{'alias': 'galleries', 'title': 'Art Galleries'}]
Index : 29282, Value : [{'alias': 'engraving', 'title': 'Engraving'}, {'alias': 'giftshops', 'title': 'Gift Shops'}]
Index : 29283, Value : [{'alias': 'landmarks', 'title': 'Landmarks & Historical Buildings'}]
Index : 29284, Value : [{'alias': 'florists', 'title': 'Florists'}]
Index : 29286, Value : [{'

Index : 29399, Value : [{'alias': 'drugstores', 'title': 'Drugstores'}]
Index : 29400, Value : [{'alias': 'pet_training', 'title': 'Pet Training'}, {'alias': 'petboarding', 'title': 'Pet Boarding'}]
Index : 29401, Value : [{'alias': 'massage', 'title': 'Massage'}]
Index : 29402, Value : [{'alias': 'weightlosscenters', 'title': 'Weight Loss Centers'}]
Index : 29403, Value : [{'alias': 'othersalons', 'title': 'Nail Salons'}]
Index : 29404, Value : [{'alias': 'retirement_homes', 'title': 'Retirement Homes'}]
Index : 29405, Value : [{'alias': 'petstore', 'title': 'Pet Stores'}, {'alias': 'pet_training', 'title': 'Pet Training'}, {'alias': 'groomer', 'title': 'Pet Groomers'}]
Index : 29406, Value : [{'alias': 'autoglass', 'title': 'Auto Glass Services'}, {'alias': 'windshieldinstallrepair', 'title': 'Windshield Installation & Repair'}]
Index : 29407, Value : [{'alias': 'realestatesvcs', 'title': 'Real Estate Services'}]
Index : 29408, Value : [{'alias': 'laundryservices', 'title': 'Laundry 

Index : 29535, Value : [{'alias': 'isps', 'title': 'Internet Service Providers'}, {'alias': 'mobilephones', 'title': 'Mobile Phones'}, {'alias': 'telecommunications', 'title': 'Telecommunications'}]
Index : 29536, Value : [{'alias': 'galleries', 'title': 'Art Galleries'}]
Index : 29537, Value : [{'alias': 'adultentertainment', 'title': 'Adult Entertainment'}]
Index : 29539, Value : [{'alias': 'photographers', 'title': 'Photographers'}]
Index : 29540, Value : [{'alias': 'womenscloth', 'title': "Women's Clothing"}]
Index : 29541, Value : [{'alias': 'furniture', 'title': 'Furniture Stores'}]
Index : 29543, Value : [{'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 29546, Value : [{'alias': 'swimminglessons', 'title': 'Swimming Lessons/Schools'}]
Index : 29547, Value : [{'alias': 'framing', 'title': 'Framing'}]
Index : 29549, Value : [{'alias': 'boating', 'title': 'Boating'}]
Index : 29550, Value : [{'alias': 'festivals', 'title': 'Festivals'}]
Index : 29552, Value : [{'alias': 'dr

Index : 29690, Value : [{'alias': 'sewingalterations', 'title': 'Sewing & Alterations'}, {'alias': 'laundryservices', 'title': 'Laundry Services'}]
Index : 29691, Value : [{'alias': 'marketing', 'title': 'Marketing'}]
Index : 29693, Value : [{'alias': 'outlet_stores', 'title': 'Outlet Stores'}]
Index : 29694, Value : [{'alias': 'generaldentistry', 'title': 'General Dentistry'}]
Index : 29695, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}]
Index : 29696, Value : [{'alias': 'homeappliancerepair', 'title': 'Appliances & Repair'}]
Index : 29697, Value : [{'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 29698, Value : [{'alias': 'nightlife', 'title': 'Nightlife'}]
Index : 29699, Value : [{'alias': 'homedecor', 'title': 'Home Decor'}]
Index : 29700, Value : [{'alias': 'shoppingcenters', 'title': 'Shopping Centers'}]
Index : 29701, Value : [{'alias': 'jewelry', 'title': 'Jewelry'}]
Index : 29703, Value : [{'alias': 'grocery', 'title': 'Grocery'}, {'alias': 'organic_stores'

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
#Checking business name
df_business['name'][31600]

'Souvlaki Kalamaki'

In [10]:
# Drop duplicate values in df_business
df_business_clean = df_business_Restaurant.drop_duplicates(subset ="business_id", 
                     keep = False)

In [11]:
# Rename Month as Day and Day as Month(error occurs on original data)
df_toronto_review=df_toronto_review.rename(columns = {'Month':'new_Day'})
df_toronto_review=df_toronto_review.rename(columns = {'Day':'Month'})
df_toronto_review=df_toronto_review.rename(columns = {'new_Day':'Day'})

In [12]:
# df_toronto_review.shape

In [13]:
# df_business_clean.shape

In [14]:
# inner join two full dataset
df = pd.merge(df_toronto_review, df_business_clean, on='business_id', how='inner')

In [15]:
df.shape

(510285, 39)

In [16]:
# merge the date and turn it into string
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])
df['date'] = df['date'].astype(str)

In [17]:
# rename the columns to match the algorithm 
df.rename(columns={'rating_x': 'stars','rating_y': 'business_stars','review_text': 'text',}, inplace=True)

In [18]:
# get rid of the rows without user_id
df = df[~df["user_id"].isnull()]

In [19]:
pd.set_option('display.max_columns', None)
df

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,categories,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date
0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon ...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-08-23
1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't bus...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-10-02
2,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,49,"['11', '6', '2016']",oHatxr1JfRewl0g49--C7g,en,{Grapefruit & Yakult Green Tea with Aloe Jelly...,"[0, 0, 0]",C91it8b3odRg0503asUaJA,"Markham, Canada",0.0,False,2016,6,11,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-11-06
3,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,164,"['9', '25', '2016']",4bnmXCDq3T7kLtTzesT_Cw,en,Saw this newly opened bubble tea shop and want...,"[0, 0, 0]",dAB-v4B-5_8QgTcoVSnZOw,"Vaughan, Canada",0.0,False,2016,25,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-25
4,10,10,Xo1LNzhnwE-ilqsM3ybs9Q,121,False,[],[],False,39.0,5.0,27,"['9', '6', '2016']",WgSTb6L_OMcTeUq5vKYmxA,en,Walked by pmall and the poster of the salty ch...,"[2, 2, 1]",UxwiJpYSqOZJQzUMF6Xl6A,"Toronto, Canada",3.0,False,2016,6,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-06
5,11,11,Xo1LNzhnwE-ilqsM3ybs9Q,139,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/5GBq...,False,181.0,3.0,81,"['2', '21', '2017']",bZzY0Oxo0tqfDgWC20tRCA,en,I got the mango mat

In [20]:
len(df)

413614

### Data Processing

In [21]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')